In [1]:
%matplotlib inline

In [2]:
import sys
import numpy as np
import pandas as pd
import re as re
import matplotlib.pyplot as plt
import xgboost as xgb

In [3]:
def myPrint(x):
    print(x.head(15))
    print(x.info())
def myPrint2(x):
    print(x)

In [4]:
test_b_voice_df = pd.read_csv('../data/Test-B/test_b_voice_df.csv', header = 0, dtype = {'opp_num':str})

# Feature 34

In [5]:
voice_opp_num = test_b_voice_df.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('voice_opp_num_')

d:\program files (x86)\python\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [6]:
print(voice_opp_num)

      voice_opp_num_count  voice_opp_num_unique_count
uid                                                  
7000                    4                           4
7001                    3                           2
7002                   65                          16
7003                   90                          22
7004                   28                           7
7005                  171                          29
7006                    2                           1
7007                    3                           3
7008                  235                          30
7009                 1308                         177
7010                    2                           2
7011                  189                          28
7012                   10                           2
7013                  109                          34
7014                  140                          36
7015                  164                          29
7016                    2   

# Feature 35

In [7]:
voice_opp_head=test_b_voice_df.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('voice_opp_head_')

d:\program files (x86)\python\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [8]:
print(voice_opp_head)

      voice_opp_head_unique_count
uid                              
7000                            4
7001                            2
7002                            9
7003                           16
7004                            6
7005                           21
7006                            1
7007                            3
7008                           18
7009                           36
7010                            2
7011                           20
7012                            2
7013                           21
7014                           18
7015                           17
7016                            1
7017                            9
7018                           21
7019                           29
7020                           12
7021                            4
7022                            1
7023                           19
7024                           34
7025                           12
7026                           19
7027          

# Feature 36

In [9]:
voice_opp_len=test_b_voice_df.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len_').fillna(0)

In [10]:
voice_opp_len.columns.name = None

In [11]:
print(voice_opp_len)

      voice_opp_len_3  voice_opp_len_5  voice_opp_len_6  voice_opp_len_7  \
uid                                                                        
7000              0.0              1.0              0.0              0.0   
7001              0.0              2.0              0.0              0.0   
7002              0.0              0.0              0.0              0.0   
7003              0.0              1.0              0.0              3.0   
7004              0.0              1.0              0.0              0.0   
7005              0.0              1.0              0.0              0.0   
7006              0.0              0.0              0.0              0.0   
7007              0.0              0.0              0.0              0.0   
7008              2.0              3.0              0.0              0.0   
7009              1.0              0.0              0.0              0.0   
7010              0.0              0.0              0.0              0.0   
7011        

# Feature 37

In [12]:
voice_call_type = test_b_voice_df.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type_').fillna(0)

In [13]:
voice_call_type.columns.name = None

In [14]:
print(voice_call_type)

      voice_call_type_1  voice_call_type_2  voice_call_type_3  \
uid                                                             
7000                4.0                0.0                0.0   
7001                3.0                0.0                0.0   
7002               65.0                0.0                0.0   
7003               89.0                0.0                1.0   
7004               25.0                3.0                0.0   
7005              156.0                4.0               11.0   
7006                2.0                0.0                0.0   
7007                3.0                0.0                0.0   
7008              234.0                1.0                0.0   
7009             1163.0               99.0               46.0   
7010                2.0                0.0                0.0   
7011              173.0                1.0               15.0   
7012               10.0                0.0                0.0   
7013               81.0  

# Feature 38

In [15]:
voice_in_out = test_b_voice_df.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_in_out_').fillna(0)

In [16]:
voice_in_out.columns.name = None

In [17]:
print(voice_in_out)

      voice_in_out_0  voice_in_out_1
uid                                 
7000             1.0             3.0
7001             0.0             3.0
7002            29.0            36.0
7003            30.0            60.0
7004            19.0             9.0
7005            62.0           109.0
7006             1.0             1.0
7007             2.0             1.0
7008           163.0            72.0
7009           683.0           625.0
7010             2.0             0.0
7011            60.0           129.0
7012             1.0             9.0
7013            62.0            47.0
7014            65.0            75.0
7015            66.0            98.0
7016             0.0             2.0
7017           476.0           434.0
7018            32.0            68.0
7019           186.0           170.0
7020            24.0            30.0
7021             4.0             1.0
7022             3.0            27.0
7023           181.0           222.0
7024           316.0           239.0
7

# Feature 39

In [18]:
test_b_1 = test_b_voice_df[test_b_voice_df['in_out'] == 0]
test_b_2 = test_b_voice_df[test_b_voice_df['in_out'] == 1]

In [19]:
t1 = test_b_1['uid'].value_counts()
t2 = test_b_2['uid'].value_counts()
dict = t1.to_dict()
tt1 = pd.Series(index = range(7000, 10000), data = dict)
tt1 = tt1.fillna(0)
dict = t2.to_dict()
tt2 = pd.Series(index = range(7000, 10000), data = dict)
tt2 = tt2.fillna(0)
tt3 = tt1 / (tt1 + tt2)

In [20]:
print(tt3)

7000    0.250000
7001    0.000000
7002    0.446154
7003    0.333333
7004    0.678571
7005    0.362573
7006    0.500000
7007    0.666667
7008    0.693617
7009    0.522171
7010    1.000000
7011    0.317460
7012    0.100000
7013    0.568807
7014    0.464286
7015    0.402439
7016    0.000000
7017    0.523077
7018    0.320000
7019    0.522472
7020    0.444444
7021    0.800000
7022    0.100000
7023    0.449132
7024    0.569369
7025    0.424658
7026    0.453237
7027    0.382353
7028    0.484127
7029    0.243094
          ...   
9970    0.500000
9971    0.202703
9972    0.381403
9973    0.360360
9974    0.487179
9975    0.500000
9976    0.443609
9977    0.612963
9978    0.666667
9979    0.437500
9980    0.621622
9981    0.230769
9982    0.853659
9983    0.507109
9984    0.623482
9985    0.702703
9986    0.406977
9987    0.294118
9988         NaN
9989    0.786667
9990    0.259259
9991    0.508143
9992    0.288462
9993    0.690476
9994    0.595588
9995    0.443902
9996    0.333333
9997    0.4000

In [21]:
tt3 = tt3.fillna(0.5)

In [24]:
voice_in_out_rate = pd.DataFrame(tt3)
voice_in_out_rate.columns = ['in_out_rate']

In [25]:
print(voice_in_out_rate)

      in_out_rate
7000     0.250000
7001     0.000000
7002     0.446154
7003     0.333333
7004     0.678571
7005     0.362573
7006     0.500000
7007     0.666667
7008     0.693617
7009     0.522171
7010     1.000000
7011     0.317460
7012     0.100000
7013     0.568807
7014     0.464286
7015     0.402439
7016     0.000000
7017     0.523077
7018     0.320000
7019     0.522472
7020     0.444444
7021     0.800000
7022     0.100000
7023     0.449132
7024     0.569369
7025     0.424658
7026     0.453237
7027     0.382353
7028     0.484127
7029     0.243094
...           ...
9970     0.500000
9971     0.202703
9972     0.381403
9973     0.360360
9974     0.487179
9975     0.500000
9976     0.443609
9977     0.612963
9978     0.666667
9979     0.437500
9980     0.621622
9981     0.230769
9982     0.853659
9983     0.507109
9984     0.623482
9985     0.702703
9986     0.406977
9987     0.294118
9988     0.500000
9989     0.786667
9990     0.259259
9991     0.508143
9992     0.288462
9993     0

# Feature 40

In [27]:
def do(x):
    start_day = int(x['start_time'] % 100000000 / 1000000)
    start_hour = int(x['start_time'] % 1000000 / 10000)
    start_min = int(x['start_time'] % 10000 / 100)
    start_sec = int(x['start_time'] % 100)
    start = (start_day * 24 * 60 + start_hour * 60 + start_min) * 60 + start_sec
    
    end_day = int(x['end_time'] % 100000000 / 1000000)
    end_hour = int(x['end_time'] % 1000000 / 10000)
    end_min = int(x['end_time'] % 10000 / 100)
    end_sec = int(x['end_time'] % 100)
    end = (end_day * 24 * 60 + end_hour * 60 + end_min) * 60 + end_sec
    
    # duration in seconds
    return end - start + 1
tmp = test_b_voice_df
test_b_voice_df.loc[:, 'dura_second'] = tmp.apply(do, axis = 1)

In [29]:
test_b_1 = test_b_voice_df[test_b_voice_df['in_out'] == 0]
test_b_2 = test_b_voice_df[test_b_voice_df['in_out'] == 1]

In [30]:
g = test_b_1[test_b_1['dura_second'] <= 10]

In [33]:
cnt = g['uid'].value_counts().sort_index()

In [42]:
cnt.index.name = 'uid'
voice_short_dura_count = pd.DataFrame(cnt)
voice_short_dura_count.columns = ['voice_short_dura_count']

In [44]:
print(voice_short_dura_count)

      voice_short_dura_count
uid                         
7000                       1
7002                       1
7005                       2
7007                       1
7008                       6
7009                      10
7011                       7
7013                       1
7014                       2
7015                       2
7017                      53
7018                       3
7019                       4
7023                      19
7024                      14
7025                       2
7026                       5
7028                      17
7029                       7
7030                       1
7033                       1
7036                       1
7037                       8
7038                       1
7040                       6
7041                       1
7044                      21
7045                      17
7047                       1
7049                       1
...                      ...
9959                       4
9961          

# Feature 41

In [46]:
def do(x):
    return int(x['start_time'] / 1000000)
tmp = test_b_voice_df
test_b_voice_df.loc[:, 'date'] = tmp.apply(do, axis = 1)

In [47]:
print(test_b_voice_df[['uid', 'start_time', 'date']])

         uid  start_time  date
0       8925    12122145    12
1       8925    12114240    12
2       8884    12150612    12
3       8884    12144742    12
4       8884    12144405    12
5       8884    12145820    12
6       8884    12145405    12
7       8884    12150840    12
8       8884    12145531    12
9       8884    12151209    12
10      8884    12145246    12
11      8884    12150001    12
12      8884    12063616    12
13      8884    12152704    12
14      8884    12144551    12
15      8884    12142700    12
16      8884    12150359    12
17      8884    12145021    12
18      8884    12151520    12
19      8884    12150225    12
20      8884    12064522    12
21      8884    12060646    12
22      8884    12150039    12
23      8884    12151403    12
24      8884    12081926    12
25      8884    12183057    12
26      8884    12201130    12
27      8884    12182845    12
28      8884    12203911    12
29      9309    12100346    12
...      ...         ...   ...
609545  

In [48]:
voice_date = test_b_voice_df.groupby(['uid','date'])['uid'].count().unstack().add_prefix('date_').fillna(0)

In [49]:
print(voice_date)

date  date_1  date_2  date_3  date_4  date_5  date_6  date_7  date_8  date_9  \
uid                                                                            
7000     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
7001     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
7002     0.0     0.0     0.0     3.0     1.0     2.0     0.0     3.0     1.0   
7003     2.0     3.0     0.0     1.0     1.0     0.0     6.0     7.0     1.0   
7004     1.0     3.0     2.0     1.0     3.0     0.0     0.0     0.0     0.0   
7005    13.0     9.0     4.0     1.0     5.0     4.0     0.0     0.0     4.0   
7006     0.0     0.0     0.0     2.0     0.0     0.0     0.0     0.0     0.0   
7007     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
7008     2.0     9.0     4.0    10.0     7.0     1.0     2.0     4.0     3.0   
7009    32.0    43.0    32.0    39.0    36.0    34.0    27.0    35.0    19.0   
7010     0.0     0.0     0.0     0.0    

# Merge Feature 34 to 41

In [50]:
agg = voice_opp_num
agg = agg.join(voice_opp_len)
agg = agg.join(voice_opp_head)
agg = agg.join(voice_call_type)
agg = agg.join(voice_in_out)
agg = agg.join(voice_in_out_rate)
agg = agg.join(voice_short_dura_count)
agg = agg.join(voice_date)

In [57]:
agg = agg.drop(u'voice_opp_len_18', axis = 1)

In [59]:
print(agg)

      voice_opp_num_count  voice_opp_num_unique_count  voice_opp_len_3  \
uid                                                                      
7000                    4                           4              0.0   
7001                    3                           2              0.0   
7002                   65                          16              0.0   
7003                   90                          22              0.0   
7004                   28                           7              0.0   
7005                  171                          29              0.0   
7006                    2                           1              0.0   
7007                    3                           3              0.0   
7008                  235                          30              2.0   
7009                 1308                         177              1.0   
7010                    2                           2              0.0   
7011                  189             

In [60]:
dict = agg.to_dict()
agg2 = pd.DataFrame(index = range(7000, 10000), data = dict)

In [61]:
agg2 = agg2.fillna(0)
agg2 = agg2.astype('int64')

In [62]:
agg2.to_csv('../data/Test-B/test_b_feature_34to41.csv', index=True, header=None)